# Session 9: Synthetic Data Generation and RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow, and use it to evaluate and iterate on a RAG pipeline with LangSmith!

**Learning Objectives:**
- Understand Ragas' knowledge graph-based synthetic data generation workflow
- Generate synthetic test sets with different query synthesizer types
- Load synthetic data into LangSmith for evaluation
- Evaluate a RAG chain using LangSmith evaluators
- Iterate on RAG pipeline parameters and measure the impact

## Table of Contents:

- **Breakout Room #1:** Synthetic Data Generation with Ragas
  - Task 1: Dependencies and API Keys
  - Task 2: Data Preparation and Knowledge Graph Construction
  - Task 3: Generating Synthetic Test Data
  - Question #1 & Question #2
  - 🏗️ Activity #1: Custom Query Distribution

- **Breakout Room #2:** RAG Evaluation with LangSmith
  - Task 4: LangSmith Dataset Setup
  - Task 5: Building a Basic RAG Chain
  - Task 6: Evaluating with LangSmith
  - Task 7: Modifying the Pipeline and Re-Evaluating
  - Question #3 & Question #4
  - 🏗️ Activity #2: Analyze Evaluation Results

---
# 🤝 Breakout Room #1
## Synthetic Data Generation with Ragas

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [11]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/rahul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rahul/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [13]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [14]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using two complementary guides — a Health & Wellness Guide covering exercise, nutrition, sleep, and stress management, and a Mental Health & Psychology Handbook covering mental health conditions, therapeutic approaches, resilience, and daily mental health practices. The topical overlap between documents helps RAGAS build rich cross-document relationships in the knowledge graph.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [15]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader("data/", glob="*.txt", loader_cls=TextLoader)
docs = loader.load()
print(f"Loaded {len(docs)} documents: {[d.metadata['source'] for d in docs]}")

Loaded 2 documents: ['data/MentalHealthGuide.txt', 'data/HealthWellnessGuide.txt']


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [16]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [17]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [18]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [19]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 9, relationships: 16)

We can save and load our knowledge graphs as follows.

In [20]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 9, relationships: 16)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [21]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [23]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

## ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### Answer:
SingleHopSpecificQuerySynthesizer --> This is for single fact based retrieval queries. e.g. what's the capital for France?  This would result in single retrieval without any need to complex joins.
MultiHopAbstractQuerySynthesizer --> This is for complex reasoing/conceptual queries. e.g How Wholefood acquisition informed Amazon Strategy. This required resaoning beyond fact retrievals.
MultiHopSpecificQuerySynthesizer --> This is for complex fact based queries -- requires retreival from multiple documents to answer. e.g. When Wholefood was acquired by Amazon, at that timeframe did Walmart made any similar acquisitions?


Finally, we can use our `TestSetGenerator` to generate our testset!

In [24]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Whaat is the impportance of Mental Health in o...,[The Mental Health and Psychology Handbook A P...,"Mental health encompasses our emotional, psych...",single_hop_specifc_query_synthesizer
1,What is MBSR?,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Mindfulness-Based Stress Reduction (MBSR) is a...,single_hop_specifc_query_synthesizer
2,Can you explain how CBT-I functions as a treat...,[Write letters to or from your future self Jou...,"CBT-I, or Cognitive Behavioral Therapy for Ins...",single_hop_specifc_query_synthesizer
3,What role do Marriage and Family Therapists pl...,[social interactions How to set and maintain b...,The context mentions Marriage and Family Thera...,single_hop_specifc_query_synthesizer
4,Can you explain the role of macronutrients in ...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Macronutrients are key components of a balance...,single_hop_specifc_query_synthesizer
5,How can behavioral techniques like journaling ...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,"Behavioral techniques such as journaling, incl...",multi_hop_abstract_query_synthesizer
6,Signs needing pro mental health help and types...,[<1-hop>\n\nsocial interactions How to set and...,Signs indicating the need for professional men...,multi_hop_abstract_query_synthesizer
7,How can incorporating mindfulness and meditati...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Incorporating mindfulness and meditation pract...,multi_hop_abstract_query_synthesizer
8,How do the concepts discussed in Chapter 11 on...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Chapter 11 emphasizes stress reduction techniq...,multi_hop_specific_query_synthesizer
9,How can Cognitive Behavioral Therapy (CBT) and...,[<1-hop>\n\nWrite letters to or from your futu...,Cognitive Behavioral Therapy (CBT) focuses on ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [25]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [26]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wha is the United States's role in mental heal...,[The Mental Health and Psychology Handbook A P...,The context provided does not specify the role...,single_hop_specifc_query_synthesizer
1,What is MBCT?,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Mindfulness-Based Cognitive Therapy (MBCT) com...,single_hop_specifc_query_synthesizer
2,How does oxytocin influence social connection ...,[Write letters to or from your future self Jou...,The provided context discusses the importance ...,single_hop_specifc_query_synthesizer
3,Are psychiatrists the only mental health profe...,[social interactions How to set and maintain b...,"Yes, the context mentions psychiatrists as one...",single_hop_specifc_query_synthesizer
4,how social connection and its importance for m...,[<1-hop>\n\nWrite letters to or from your futu...,the context explains that social connection is...,multi_hop_abstract_query_synthesizer
5,What are the recommended physical activity gui...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,The Personal Wellness Guide recommends adults ...,multi_hop_abstract_query_synthesizer
6,"How can journaling practices, such as self-ref...",[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,Journaling practices like self-reflection and ...,multi_hop_abstract_query_synthesizer
7,How do dietary patterns such as Mediterranean ...,[<1-hop>\n\nWrite letters to or from your futu...,The context highlights that dietary patterns s...,multi_hop_abstract_query_synthesizer
8,How can Cognitive Behavioral Therapy (CBT) and...,[<1-hop>\n\nWrite letters to or from your futu...,Integrating Cognitive Behavioral Therapy (CBT)...,multi_hop_specific_query_synthesizer
9,How can Cognitive Behavioral Therapy (CBT) and...,[<1-hop>\n\nWrite letters to or from your futu...,Cognitive Behavioral Therapy (CBT) is an effec...,multi_hop_specific_query_synthesizer


## ❓ Question #2:

Ragas offers both an "unrolled" (manual) approach and an "abstracted" (automatic) approach to synthetic data generation. What are the trade-offs between these two approaches? When would you choose one over the other?

##### Answer:
In unrolled, I had the advantage of specifying my query types. I was able to pick 50% of simple, 25% each of abstract and multihop type. I don't see that options here. On the flip side, in case I don't care about such distribution, abstracted approach is much faster. 
Another key point is, we do not have to worry about building knowlegde graph with abstracted method. In most business usecases, it would likely be easier to work with abstracted approach and keep the build out simple. 

---
## 🏗️ Activity #1: Custom Query Distribution

Modify the `query_distribution` to experiment with different ratios of query types.

### Requirements:
1. Create a custom query distribution with different weights than the default
2. Generate a new test set using your custom distribution
3. Compare the types of questions generated with the default distribution
4. Explain why you chose the weights you did

In [ ]:
### YOUR CODE HERE ###
# Define a custom query distribution with different weights
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
]
# Generate a new test set and compare with the default
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()


Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How does the World Health Organization define ...,[The Mental Health and Psychology Handbook A P...,"According to the World Health Organization, me...",single_hop_specifc_query_synthesizer
1,What is DBT and how does it help in mental hea...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Dialectical Behavior Therapy (DBT) was origina...,single_hop_specifc_query_synthesizer
2,"How does zinc influence mental health, especia...",[Write letters to or from your future self Jou...,"Zinc is a nutrient found in oysters, beef, and...",single_hop_specifc_query_synthesizer
3,how sleep and sleep disorders like insomnia an...,[<1-hop>\n\nWrite letters to or from your futu...,Sleep and mental health have a bidirectional r...,multi_hop_abstract_query_synthesizer
4,"How do factors influencing mental health, such...",[<1-hop>\n\nThe Mental Health and Psychology H...,The Mental Health and Psychology Handbook expl...,multi_hop_abstract_query_synthesizer
5,how can setting boundaries and managing digita...,[<1-hop>\n\nsocial interactions How to set and...,Managing digital mental health involves settin...,multi_hop_abstract_query_synthesizer
6,Hwo can I use chaptrs 11 and 20 to improve my ...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Chapters 11 and 20 provide guidance on managin...,multi_hop_specific_query_synthesizer
7,How do Chapters 9 and 17 together inform strat...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,Chapter 9 explains that good sleep is crucial ...,multi_hop_specific_query_synthesizer
8,How can Cognitive Behavioral Therapy (CBT) and...,[<1-hop>\n\nWrite letters to or from your futu...,Cognitive Behavioral Therapy (CBT) is an effec...,multi_hop_specific_query_synthesizer
9,How can Cognitive Behavioral Therapy (CBT) and...,[<1-hop>\n\nWrite letters to or from your futu...,Cognitive Behavioral Therapy (CBT) is an effec...,multi_hop_specific_query_synthesizer


I have 3 single hop, 3 mutlihop abstract and 6 multihop queries in my testset (25%, 25% and 50%). In previous case this was 50%, 25% and 25%. 

We'll need to provide our LangSmith API key, and set tracing to "true".

---
# 🤝 Breakout Room #2
## RAG Evaluation with LangSmith

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [29]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Use Case Synthetic Data - AIE9 - {uuid.uuid4()}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [30]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [31]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [33]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [34]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="use_case_rag"
)

In [35]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [36]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [37]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [38]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [39]:
rag_chain.invoke({"question" : "What are some recommended exercises for lower back pain?"})

'Recommended exercises for lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [40]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [41]:
from openevals.llm import create_llm_as_judge
from langsmith.evaluation import evaluate

# 1. QA Correctness (replaces LangChainStringEvaluator("qa"))
qa_evaluator = create_llm_as_judge(
    prompt="You are evaluating a QA system. Given the input, assess whether the prediction is correct.\n\nInput: {inputs}\nPrediction: {outputs}\nReference answer: {reference_outputs}\n\nIs the prediction correct? Return 1 if correct, 0 if incorrect.",
    feedback_key="qa",
    model="openai:gpt-4o" ,  # pass your LangChain chat model directly
)

# 2. Labeled Helpfulness (replaces LangChainStringEvaluator("labeled_criteria"))
labeled_helpfulness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "helpfulness: Is this submission helpful to the user, "
        "taking into account the correct reference answer?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n"
        "Reference answer: {reference_outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="helpfulness",
    model="openai:gpt-4o" ,
)

# 3. Dopeness (replaces LangChainStringEvaluator("criteria"))
dopeness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "dopeness: Is this response dope, lit, cool, or is it just a generic response?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="dopeness",
    model="openai:gpt-4o" ,
)

> **Describe what each evaluator is evaluating:**
>
> - `qa_evaluator`: Whether answer is factually correct?
> - `labeled_helpfulness_evaluator`: Whether answer is helpful to user?
> - `dopeness_evaluator`: Is the answer dope/lit/cool - has a tone (e.g. BOOM.. then facts)?

## LangSmith Evaluation

In [42]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'crushing-knife-94' at:
https://smith.langchain.com/o/13656150-85dd-4f97-876b-4cfb5bff216f/datasets/22b053f2-2e2f-44e8-92d9-48377bfb3a71/compare?selectedSessions=7352c0b8-63a2-4ed6-b00b-9fb0eeed1695




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.qa,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,what chapter 4 and 14 say about healthy habits...,Chapter 4 is not mentioned in the provided con...,None,Chapter 4 discusses the fundamentals of health...,False,False,False,3.005612,cad4e698-b87c-4f92-99d5-2c5ee97d5025,019c692b-be05-7f62-8a0f-9cd552aafc61
1,What chapter talks about habit 13 and how it h...,Chapter 13 discusses the science of habit form...,None,Habit 13 explains how habits form through the ...,False,True,False,2.237582,e3f2be8c-a5e3-4216-bc16-b05d79b1a425,019c692b-f684-7251-8fac-c24e8a4ac96d
2,How can Cognitive Behavioral Therapy (CBT) and...,"Based on the provided context, Cognitive Behav...",None,Cognitive Behavioral Therapy (CBT) is an effec...,True,True,False,3.580854,1328abcc-a1bf-436d-b951-f94f78ff0905,019c692c-4b4d-71a2-93ff-e45615a8c23a
3,How can Cognitive Behavioral Therapy (CBT) and...,"Based on the provided context, Cognitive Behav...",None,Integrating Cognitive Behavioral Therapy (CBT)...,True,True,True,5.759871,5150a2fb-c970-484a-8f93-b5801fe6ca3b,019c692c-857a-7a22-9ef5-c778c1099238
4,How do dietary patterns such as Mediterranean ...,Based on the provided context:\n\nDietary patt...,None,The context highlights that dietary patterns s...,True,True,True,4.643254,231d556d-26f3-4419-9b7b-0df931242bb3,019c692c-c9f5-7902-8443-06a47cb69091
5,"How can journaling practices, such as self-ref...","Based on the provided context, journaling prac...",None,Journaling practices like self-reflection and ...,True,True,True,3.683614,be150bce-6c70-4a4d-a304-fd7fc1cade0c,019c692d-08e6-7021-96ad-d3066709553f
6,What are the recommended physical activity gui...,The recommended physical activity guidelines a...,None,The Personal Wellness Guide recommends adults ...,True,True,False,3.789290,e8ef7ed0-b384-452d-9ab9-8e55991edb72,019c692d-3d4c-7c23-8f65-3b6b57dfec01
7,how social connection and its importance for m...,Social connection is essential for mental heal...,None,the context explains that social connection is...,True,True,True,5.663769,4bcb8fd3-e306-4f2f-9f9d-ba61f3d51a67,019c692d-7c3b-7173-a70f-56179fbc9336
8,Are psychiatrists the only mental health profe...,"No, psychiatrists are not the only mental heal...",None,"Yes, the context mentions psychiatrists as one...",False,False,False,1.730412,6c8a389b-a3d7-41a6-943b-9e997b42022c,019c692d-b4ee-7883-83c3-a528c7507477
9,How does oxytocin influence social connection ...,"According to the provided context, oxytocin is...",None,The provided context discusses the importance ...,True,True,False,1.188567,4b2fb6fa-7f3c-4e80-90b7-9d47543e1650,019c692d-d845-7613-a36b-4ff2364b1c08


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [43]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [44]:
rag_documents = docs

In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

## ❓ Question #3:

Why would modifying our chunk size modify the performance of our application?

##### Answer:
Too large → noisy context
Too small → fragmented reasoning

In our case, increasing chunk size resulting in bigger chunks and likely produced document which was not as fragmented a before thereby making the context much more useful and complete. 

In [49]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## ❓ Question #4:

Why would modifying our embedding model modify the performance of our application?

##### Answer:
large embedding model produced more dimension and hence can capture subtle similariy and hence may be more benefitial for multihop queries. Whereas smalll embedding model is more for very simple fact based retreival. Depeding on kind of queries needs to be tackled, embedding is important consideration to improve retrieval accuracy for multihop.

In [50]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [51]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [52]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [53]:
dopeness_rag_chain.invoke({"question" : "How can I improve my sleep quality?"})

'Yo, wanna level up your sleep game like a boss? Here’s the cheat code straight from the sleep sages:\n\n1. **Lock down a consistent sleep schedule**—even weekends are sacred. Your body’s internal clock loves the rhythm.\n2. **Ritualize your bedtime:** think cozy vibes like reading a dope book, gentle stretches, or a warm bath to hit that chill mode.\n3. **Turn your bedroom into a sleep fortress:** keep it cool (65-68°F / 18-20°C), blackout those windows, and drown out distractions with white noise or earplugs.\n4. **Ban the blue screen monsters** at least 1-2 hours before bed. Your brain needs darkness, not that smartphone glow.\n5. **Ditch caffeine after 2 PM** unless you wanna be wide-eyed at midnight.\n6. **Move that body regularly**, but don’t sprint to bed—exercise too close to lights out can mess you up.\n7. **Keep alcohol and heavy meals out of your pre-sleep zone** to avoid tossing and turning.\n\nAnd bonus: invest in a mattress and pillows that feel like a cloud hugging you t

Finally, we can evaluate the new chain on the same test set!

In [54]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'giving-jump-91' at:
https://smith.langchain.com/o/13656150-85dd-4f97-876b-4cfb5bff216f/datasets/22b053f2-2e2f-44e8-92d9-48377bfb3a71/compare?selectedSessions=02bf5026-aae7-46af-bd08-7a2073dab1f5




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.qa,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,what chapter 4 and 14 say about healthy habits...,"Yo, let’s crank this up to legend level! Chapt...",None,Chapter 4 discusses the fundamentals of health...,True,True,True,7.060394,cad4e698-b87c-4f92-99d5-2c5ee97d5025,019c6936-3619-7420-9cae-d300c70fc3ad
1,What chapter talks about habit 13 and how it h...,"Yo, here’s the scoop straight from the wellnes...",None,Habit 13 explains how habits form through the ...,False,False,True,3.192393,e3f2be8c-a5e3-4216-bc16-b05d79b1a425,019c6936-7cd6-7ed0-89cd-37dd6ff763eb
2,How can Cognitive Behavioral Therapy (CBT) and...,"Alright, buckle up for this mind-and-snooze co...",None,Cognitive Behavioral Therapy (CBT) is an effec...,True,True,True,6.102511,1328abcc-a1bf-436d-b951-f94f78ff0905,019c6936-b7d6-7e13-90f9-e56b9b141ee4
3,How can Cognitive Behavioral Therapy (CBT) and...,"Alright, buckle up for a mental-health glow-up...",None,Integrating Cognitive Behavioral Therapy (CBT)...,True,True,True,5.206990,5150a2fb-c970-484a-8f93-b5801fe6ca3b,019c6936-f8f1-70f3-aaf5-ed277038d289
4,How do dietary patterns such as Mediterranean ...,"Alright, buckle up for this mental health and ...",None,The context highlights that dietary patterns s...,True,True,True,6.908488,231d556d-26f3-4419-9b7b-0df931242bb3,019c6937-4e45-7b62-bd32-8b53e3db0faa
5,"How can journaling practices, such as self-ref...","Oh, you’re diving deep into the mental health ...",None,Journaling practices like self-reflection and ...,True,True,True,5.169840,be150bce-6c70-4a4d-a304-fd7fc1cade0c,019c6937-8c87-7571-b0f1-4e8e501dad60
6,What are the recommended physical activity gui...,"Oh snap, you’re diving into the ultimate welln...",None,The Personal Wellness Guide recommends adults ...,True,True,True,12.369106,e8ef7ed0-b384-452d-9ab9-8e55991edb72,019c6937-ca84-7993-995f-2d87ab7de49f
7,how social connection and its importance for m...,"Alright, buckle up for some mind-blowing menta...",None,the context explains that social connection is...,True,True,True,4.937219,4bcb8fd3-e306-4f2f-9f9d-ba61f3d51a67,019c6938-2078-7873-98de-c757a7b6879f
8,Are psychiatrists the only mental health profe...,"Nah, fam! Psychiatrists aren’t solo in the men...",None,"Yes, the context mentions psychiatrists as one...",True,False,True,2.651907,6c8a389b-a3d7-41a6-943b-9e997b42022c,019c6938-668e-7ad3-b1e5-9848b73bac87
9,How does oxytocin influence social connection ...,"Alright, here’s the rad scoop straight from th...",None,The provided context discusses the importance ...,True,True,True,3.611990,4b2fb6fa-7f3c-4e80-90b7-9d47543e1650,019c6938-a72a-78d3-8a96-2c5faff375e2


---
## 🏗️ Activity #2: Analyze Evaluation Results

Provide a screenshot of the difference between the two chains in LangSmith, and explain why you believe certain metrics changed in certain ways.

##### Answer:
*Your answer here*

First Case: Small Embedding Model and No DOPENESS.  Filepath: 09_Synthetic_Data_Generation_and_LangSmith/Small_emnbedding_no_dope.png 
Second Case: Large Embedding Model and Full on DOPENESS.  Filepath: 09_Synthetic_Data_Generation_and_LangSmith/Large_emnbedding_yes_dope.png

Observations:

Retrival Quality improved. QA score jumped from : 0.75 → 0.83 <br>
Dopeness major improvement observed. Score jumped from: 0.33 to 1. \n <br>
Helpfulness didn't see any impact -- becasue parameters we modifying only updated factual accuracy and increased dopeness.




---
## Summary

In this session, we:

1. **Generated synthetic test data** using Ragas' knowledge graph-based approach
2. **Explored query synthesizers** for creating diverse question types
3. **Loaded synthetic data** into a LangSmith dataset for evaluation
4. **Built and evaluated a RAG chain** using LangSmith evaluators
5. **Iterated on the pipeline** by modifying chunk size, embedding model, and prompt — then measured the impact

### Key Takeaways:

- **Synthetic data generation** is critical for early iteration — it provides high-quality signal without manually creating test data
- **LangSmith evaluators** enable systematic comparison of pipeline versions
- **Small changes matter** — chunk size, embedding model, and prompt modifications can significantly affect evaluation scores